In [125]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [99]:
df = pd.read_csv("../data/merge_all/merge_all.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8370 entries, 2000-01-01 to 2022-11-30
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   자동차용 경유 가격 (원)  8370 non-null   float64
 1   전월비(%)          8370 non-null   float64
 2   평균              8370 non-null   float64
 3   거래량             8370 non-null   float64
dtypes: float64(4)
memory usage: 327.0+ KB


In [98]:
# df["거래년월일"] = pd.to_datetime(df["거래년월일"],format="%Y-%m-%d")
# df.info()

In [100]:
feature = df[["자동차용 경유 가격 (원)", "전월비(%)", "거래량"]]
target = df["평균"]

In [101]:
std = StandardScaler()
std.fit(feature)
feature_std = std.transform(feature)

In [102]:
feature_std

array([[-1.88976635,  0.6975613 , -1.06460874],
       [-1.88976635,  0.6975613 , -1.06460874],
       [-1.88976635,  0.6975613 , -1.06460874],
       ...,
       [ 1.61629349, -1.65639961, -0.81312453],
       [ 1.61629349, -1.65639961, -1.12838427],
       [ 1.61629349, -1.65639961, -1.02024603]])

In [87]:
# feature_std = pd.DataFrame(feature_std, columns = ["자동차용 경유 가격 (원)", "전월비(%)", "거래량"])


In [103]:
# feature_std["거래년월일"] = df["거래년월일"]
# feature_std

In [104]:
# feature_std.info()

In [105]:
# feature_std = feature_std.values
# type(feature_std)

In [106]:
std2 = StandardScaler()
std2.fit(target.values.reshape(-1,1))
target_std = std2.transform(target.values.reshape(-1,1))
# target_std.shape

In [107]:
# target_std = target_std.reshape(-1)

In [108]:
target_std.shape

(8370, 1)

In [114]:
X_train, X_test = feature_std[:1674], feature_std[1674:]
y_train, y_test = target_std[:1674], target_std[1674:]

In [140]:
model = Sequential()
model.add(LSTM(units=3, input_shape=(X_train.shape[1],1), activation="tanh", ))
model.add(Dense(1))
model.compile(loss="mean_squared_error",optimizer = "adam", metrics=["mse"])

ValueError: Input 0 of layer "lstm_33" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 3)

In [136]:
model.fit(X_train, y_train ,epochs=300, verbose=2)

Epoch 1/300
53/53 - 5s - loss: 0.4490 - mse: 0.4490 - 5s/epoch - 101ms/step
Epoch 2/300
53/53 - 0s - loss: 0.3470 - mse: 0.3470 - 142ms/epoch - 3ms/step
Epoch 3/300
53/53 - 0s - loss: 0.2915 - mse: 0.2915 - 163ms/epoch - 3ms/step
Epoch 4/300
53/53 - 0s - loss: 0.2678 - mse: 0.2678 - 157ms/epoch - 3ms/step
Epoch 5/300
53/53 - 0s - loss: 0.2593 - mse: 0.2593 - 148ms/epoch - 3ms/step
Epoch 6/300
53/53 - 0s - loss: 0.2553 - mse: 0.2553 - 141ms/epoch - 3ms/step
Epoch 7/300
53/53 - 0s - loss: 0.2538 - mse: 0.2538 - 143ms/epoch - 3ms/step
Epoch 8/300
53/53 - 0s - loss: 0.2530 - mse: 0.2530 - 142ms/epoch - 3ms/step
Epoch 9/300
53/53 - 0s - loss: 0.2525 - mse: 0.2525 - 142ms/epoch - 3ms/step
Epoch 10/300
53/53 - 0s - loss: 0.2520 - mse: 0.2520 - 136ms/epoch - 3ms/step
Epoch 11/300
53/53 - 0s - loss: 0.2516 - mse: 0.2516 - 138ms/epoch - 3ms/step
Epoch 12/300
53/53 - 0s - loss: 0.2513 - mse: 0.2513 - 151ms/epoch - 3ms/step
Epoch 13/300
53/53 - 0s - loss: 0.2510 - mse: 0.2510 - 169ms/epoch - 3ms/s

In [137]:
model.evaluate(X_test, y_test)

210/210 [==============================] - 1s 2ms/step - loss: 1.2818 - mse: 1.2818


[1.2818166017532349, 1.2818166017532349]

In [138]:
y_pred = model.predict(X_test)
print("r2 score :", r2_score(y_test, y_pred))

r2 score : -0.16600797285735958


-------------------------------------------------

In [ ]:
# 데이터 스케일링
scaler = MinMaxScaler()
scaler.fit(feature)
feature = scaler.transform(feature)
scaler.fit(target)
target = scaler.transform(target)

# 데이터셋 생성
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# 데이터셋 생성
look_back = 1
trainX, trainY = create_dataset(feature, look_back)
testX, testY = create_dataset(target, look_back)

# 데이터셋 형태 변환
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# 예측
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# 데이터 스케일링 복원
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# 모델 평가
trainScore = model.evaluate(trainX, trainY, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, np.sqrt(trainScore)))
testScore = model.evaluate(testX, testY, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, np.sqrt(testScore)))

# 예측 결과 시각화
plt.figure(figsize=(12, 9))
plt.plot(testY[0], label='actual')
plt.plot(testPredict, label='prediction')
plt.legend()
plt.show()

# 예측 결과 저장
predic_df = pd.DataFrame(testPredict)

# 예측 결과 저장
